In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import OneHotEncoder

import seaborn as sns
# загружаем библиотеку для анализа интенсивности настроения
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')
import plotly.express as px
import category_encoders as ce
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# <center> ОПИСАНИЕ И УСЛОВИЯ  соревнования
Представьте, что вы работаете датасаентистом в компании Booking. Одна из проблем компании — это нечестные отели, которые накручивают себе рейтинг. Одним из способов нахождения таких отелей является построение модели, которая предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель играет нечестно, и его стоит проверить.

Вам поставлена задача создать такую модель. Готовы приступить?

Условия соревнования:
Данное соревнование является бессрочным и доступно для всех потоков.

Срок выполнения соревнования устанавливается индивидуально в каждом потоке.

Тестовая выборка представлена в LeaderBoard целиком.

Делаем реальный ML продукт, который потом сможет нормально работать на новых данных.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
#DATA_DIR = 'data/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

# Предварительный анализ

In [ ]:
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.info()

In [ ]:
sample_submission.head(2)

In [ ]:
# для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 #  трейн
df_test['sample'] = 0 # тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

# Создание новых признаков

# **<b>1. Обработка колонки 'review_date'**

In [ ]:
# преобразовывание колонки даты в формат datetime
data['review_date'] = pd.to_datetime(data['review_date'])

# извлекаем день недели из даты
data['day_of_week'] = data['review_date'].dt.dayofweek

# извлекаем месяц из даты
data['month'] = data['review_date'].dt.month

# также извлекаем количество дней между датой проверки и датой очистки и преобразовываем к числовому типу данных
data['days_since_review'] = data['days_since_review'].apply(lambda x: x.split()[0]).astype('int')
data['days_since_review'].head()

In [ ]:
# Функция указывающая сезон года
def get_season(date):
    if date.month in [1, 2, 12]:
        return 'winter'
    if date.month in [3, 4, 5]:
        return 'spring'
    if date.month in [6, 7, 8]:
        return 'summer'
    else:
        return 'autumn'
data['season'] = data['review_date'].apply(get_season)

In [ ]:
# Визуализация полученных данных по сезонам
season_data = data['season'].value_counts()
fig = plt.figure(figsize=(5, 5))
axes = fig.add_axes([0, 0, 1, 1])
axes.pie(
    season_data,
    labels = season_data.index,
    autopct = '%.1f%%',
    explode = [0.1, 0, 0, 0]
);

# **2. Обработка адресов отелей hotel_address**

In [ ]:
#выделим страну и название города из адреса отеля
data['hotel_city'] = data['hotel_address'].apply(lambda x: x.split()[-5] if x.endswith('United Kingdom') else x.split()[-2])
data['hotel_country'] = data['hotel_address'].apply(lambda x: 'United Kingdom' if x.endswith('United Kingdom') else x.split()[-1])
data.drop('hotel_address',axis=1, inplace=True)

In [ ]:
#сводная таблица распределения городов отелей по странам
pd.pivot_table(data, values = 'sample',
               index = ['hotel_country','hotel_city'], 
               aggfunc = 'count')

Поскольку каждой стране соответствует один город, то кодирование нужно лишь одного из признаков. у
Построим график распределения отелей по странам с учетом того, что по каждому отелю в данных существует несколько отзывов.

In [ ]:
#группировка отелеи по названию и стране 
country_hotels = data.groupby(['hotel_name','hotel_country'],as_index=False)['sample'].count()
country_hotels

In [ ]:
#данные для построения графика о распределении отелей по странам
country_hotels_grouped = country_hotels.groupby('hotel_country',as_index=False)['hotel_name'].count()
country_hotels_grouped.rename(columns={'hotel_name':'hotels_number'},inplace=True)
country_hotels_grouped

In [ ]:
#построение графика
fig = px.bar(
    data_frame=country_hotels_grouped, 
    x='hotel_country',
    y='hotels_number', 
    color='hotel_country', 
    text = 'hotels_number', 
    orientation='v', 
    height=500, 
    width=1000, 
    title='Распределение отелей по странам' 
)


fig.show()

Большая часть отелей расположена во Франции и Великобритании, а так же Испании, Италии и Нидерландах.

In [ ]:
#кодировка города отеля методом однократного кодирования OneHotEncoder
encoder = ce.OneHotEncoder(cols=['hotel_city']) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data['hotel_city'])
data = pd.concat([data, type_bin], axis=1)

In [ ]:
# заполним пропуски датасета, а именно признаков географической широты и долготы, нулями с целью последующей обработки
values = {
    'lat': 0,
    'lng': 0,
}
data = data.fillna(values)

# заполним координаты географической широты отеля релеватными координатами города, в котором находится отель
def lat_fillna(x):
    lat = x[0]
    city = x[1]
    if city == 'Paris' and lat == 0:
        return 48.8567
    elif city == 'Vienna' and lat == 0:
        return 48.2000
    elif city == 'Barcelona' and lat == 0:
        return 41.3833
    elif city == 'London' and lat == 0:
        return 51.5072
    elif city == 'Milan' and lat == 0:
        return 45.4666
    elif city == 'Amsterdam' and lat == 0:
        return 52.3666
    else:
        return lat
    
data['lat'] = data[['lat', 'hotel_city']].apply(lat_fillna, axis = 1)

# заполним координаты географической долготы отеля релеватными координатами города, в котором находится отель
def lng_fillna(x):
    lng = x[0]
    city = x[1]
    if city == 'Paris' and lng == 0:
        return 2.3508
    elif city == 'Vienna' and lng == 0:
        return 16.3666
    elif city == 'Barcelona' and lng == 0:
        return 2.1833
    elif city == 'London' and lng == 0:
        return 0.1275
    elif city == 'Milan' and lng == 0:
        return 9.1833
    elif city == 'Amsterdam' and lng == 0:
        return 4.9000
    else:
        return lng
    
data['lng'] = data[['lng', 'hotel_city']].apply(lng_fillna, axis = 1)

In [ ]:
# убеждаемся в том, что все пропуски обработаны
data[data.lat.isna()]

# **<b>3. Обработка колонки 'reviewer_nationality'**

In [ ]:
# посмотрим, сколько всего уникальных признаков национальности
print(len(data['reviewer_nationality'].unique()))
print()

# выделим 10 наиболее активных национальностей
top_ten_nationalities= data['reviewer_nationality'].value_counts().nlargest(10).index
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x if x in top_ten_nationalities else 'Other')

display(data['reviewer_nationality'].head(5))

# кодируем признак национальности с помощью LabelEncoder
data['reviewer_nationality'] = LabelEncoder().fit_transform(data['reviewer_nationality'])
display(data.head(5))

In [ ]:
#данные для построения графика
bar_data = data.groupby('reviewer_nationality',as_index=False
)[['sample']].count().nlargest(15, columns=['sample'])
bar_data.rename(columns={'sample':'reviewer_number'},inplace=True)
bar_data

In [ ]:
#график распределения рецензентов по странам
fig = px.bar(
    data_frame=bar_data, 
    x="reviewer_nationality", 
    y="reviewer_number", 
    color='reviewer_nationality', 
    text = 'reviewer_number', 
    orientation='v', 
    height=500, 
    width=1000, 
    title='Распределение национальности рецензентов' 
)


fig.show()

# **4. Обработка тегов**

In [ ]:
#функциz для создания словаря с подсчетом тегов
unique_dic = {}
def make_unique_dic(x):
    x = x[3:-3].split(" ', ' ")
    for elem in x:
        if elem in unique_dic:
            unique_dic[elem] = unique_dic[elem] + 1
        else:
            unique_dic[elem] = 1
    return x

data['tags'].apply(make_unique_dic)

# получим датафрейм со всеми тегами и их частотой встречаемости
unique_tags = pd.DataFrame(unique_dic, index=['count']).transpose()
display(unique_tags)

# 5. Обработка отзывов

In [ ]:
# добавим доли позитивных и негативных слов в общем количестве слов в отзыве
data['%_positive_words_in_review'] = data['review_total_positive_word_counts'] / (data['review_total_positive_word_counts'] + data['review_total_negative_word_counts'])
data['%_negative_words_in_review'] = data['review_total_negative_word_counts'] / (data['review_total_positive_word_counts'] + data['review_total_negative_word_counts'])

# добавим доли позитивных и негативных слов в расчете на один отзыв
data['%_positive_words_per_review'] = data['review_total_positive_word_counts'] / data['total_number_of_reviews_reviewer_has_given']
data['%_negative_words_per_review'] = data['review_total_negative_word_counts'] / data['total_number_of_reviews_reviewer_has_given']

data.head(3)

In [ ]:
values = {
    '%_positive_words_in_review': data['%_positive_words_in_review'].median(),
    '%_negative_words_in_review': data['%_negative_words_in_review'].median(),
}

# заполняем пропуски в соответствии с заявленным словарем
data = data.fillna(values)

In [ ]:
data[data['%_positive_words_in_review'].isna()]

In [ ]:
data.info()

In [ ]:
# перепроверяем наличие пропусков
data.info()

In [ ]:
# анализируя отзывы, можно обратить внимание на такие слова, как "No Negative" и "No Positive".
# анализатор оценивает их как негативный и положительный отзыв соответственно.
# однако в данном контексте эти словосочетания интерпретируются в обратном смысле.

data['no_neg'] = 0
data['no_pos'] = 0

no_neg_list = ['nothing', 'no negative', 'n a', 'none', '', 'all good', 'nothing really', 'no complaints', 'nil', 'nothing at all', 'na', 'nothing to dislike', 'liked everything',
              'can t think of anything', 'everything was perfect', 'no', 'absolutely nothing', 'nothing to dislike', 'everything was great', 'nothing to complain about', 'non']

data.loc[data['negative_review'].str.lower().str.strip().isin(no_neg_list), 'no_neg'] = 1
data.loc[data['negative_review'].str.lower().str.strip().isin(['everything']), 'no_pos'] = 1
data.loc[data['negative_review'].str.lower().str.strip().isin(no_neg_list), 'negative_review'] = ''

data['negative_review'].value_counts()[:10]

In [ ]:
no_pos_list = ['no positive', 'nothing']

data.loc[data['positive_review'].str.lower().str.strip().isin(no_pos_list), 'no_pos'] = 1
data.loc[data['positive_review'].str.lower().str.strip().isin(['everything']), 'no_neg'] = 1

data['positive_review'].value_counts()[:10]

In [ ]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [ ]:
#проверка признаков на мультиколлинеарность
plt.rcParams['figure.figsize'] = (30,30)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True,cmap='coolwarm')

# **Подготовка данных к модели**

In [ ]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace = True)

# удаляем колонку с типом datetime
data.drop('review_date', axis = 1, inplace = True)

# удаляем колонки 'day_of_week' и 'month', которые уже закодированы как категориальные
data = data.drop(['day_of_week', 'month'], axis = 1)

In [ ]:
data.info()

# **Отбор признаков**

In [ ]:
# Ниже список всех столбцов-признаков для дальнейшего удобства разбиения на непрерывные и категориальные
data.columns

In [ ]:
# числовые признаки
num_cols = ['review_total_negative_word_counts', 
            'review_total_positive_word_counts',
            'total_number_of_reviews_reviewer_has_given',
            'total_number_of_reviews', 'reviewer_score',
            'additional_number_of_scoring',
            ]

# категориальные признаки
cat_cols = ['reviewer_nationality',
            'lat', 'lng', 
            'no_neg', 'no_pos',    
            'average_score',           
            ]        

In [ ]:
data[num_cols].describe()

In [ ]:
from sklearn.feature_selection import chi2 # хи-квадрат

# для оценки значимости категориальных переменных будем использовать непараметрический тест хи-квадрат.
# выведем значения хи-квадрат на графике:

y = data.query('sample == 1').drop(['sample'], axis=1).reviewer_score.values.astype('int')
X = data.query('sample == 1').drop(['sample'], axis=1)[cat_cols]

X['lng_'] = X['lng'] - X['lng'].min()
X = X.drop(['lng'], axis=1)

plt.rcParams['figure.figsize'] = (15,10)
imp_cat = pd.Series(chi2(X, y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [ ]:

from sklearn.feature_selection import f_classif # anova
# Для оценки значимости непрерывных переменных будем использовать функцию f_classif из библиотеки sklearn. Используем анализ (ANOVA).
# В качестве меры значимости будем использовать значение f-статистики - чем значение статистики выше, тем меньше вероятность того, что средние значения не отличаются, и тем важнее данный признак для нашей модели.

y = data.query('sample == 1').drop(['sample'], axis=1).reviewer_score.values.astype('int')
X = data.query('sample == 1').drop(['sample'], axis=1)[num_cols]

imp_num = pd.Series(f_classif(X, y)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')


In [ ]:
# чтобы исключить мультиколлениарность удалим признаки, сильно коррелирующий с другим признаком
pivot = data.corr()
pivot = pivot.drop('sample', axis=0)
pivot = pivot.drop('sample', axis=1)
for col in pivot:
    pivot[col] = pivot[col].apply(lambda x: np.nan if (abs(x) < 0.72 or x == 1) else x)
for col in pivot:
    pivot = pivot.dropna(how='all')
    pivot = pivot.dropna(how='all', axis='columns')
fig = px.imshow(pivot)
fig.show()

# постараемся удалить как можно меньше признаков. Сначала удаляем признаки с максимальной корреляцией
counter = 0
for lower_bound in np.linspace(0.98, 0.72, num=14):
    for col in pivot:
        if pivot[col].max() > lower_bound or pivot[col].min() < -lower_bound:
            pivot = pivot.drop(col, axis=0)
            pivot = pivot.drop(col, axis=1)
            data = data.drop(col, axis=1)
            counter += 1
print('Deleted', counter, 'columns')

In [ ]:
# Визуализация пропусков
plt.figure(figsize=(6,4))
sns.heatmap(data.isna().transpose(),
            cmap="YlGnBu",
            cbar_kws={'label': 'Пропущенные данные'})
plt.show()

In [ ]:
#провка данных на наличие значение np.inf
np.all(np.isfinite(data))

# **Обучение модели**

In [ ]:
# Выделяем тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Разделяем тренинговый датасет: 80% на обучение, 20% на валидацию
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
#Преверка размерности получившихся датасетов
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
#Импортнеобходимых библиотек:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# пишем свою функцию Mean Absolute Percentage Error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test)
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)